In [1]:
import tensorflow as tf

/Users/seongjuhyeon/workspace/tensorflowZeroToAll/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#import train set
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# To solve downloading files.
## https://github.com/tensorflow/tensorflow/issues/10779
## run the /Applications/Python 3.6/Install Certificates.command bash script to install newer certs.
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)
print(mnist.train.num_examples)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
55000


In [4]:
# 0 ~ 9까지의 숫자 갯수
nb_classes = 10

In [5]:
# MNIST Data image of shape = 28 * 28 = 784
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, nb_classes]) # one-hot으로 주어짐.

In [6]:
W1 = tf.get_variable("W1", shape=[784, 392],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable(name="b1",
                     shape=[392])
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [7]:
W2 = tf.get_variable("W2", shape=[392, 392],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable(name="b2",
                     shape=[392])
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

In [8]:
W3 = tf.get_variable("W3", shape=[392, 392],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable(name="b3",
                     shape=[392])
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

In [9]:
W4 = tf.get_variable("W4", shape=[392, nb_classes],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable(name="b4",
                     shape=[nb_classes])
hypothesis = tf.matmul(L3, W4) + b4

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

In [11]:
# Minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [16]:
# Correct Prediction
prediction = tf.argmax(hypothesis, 1)
real = tf.argmax(Y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, real), dtype=tf.float32))

In [17]:
# Prepare session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [18]:
# Slice the data, epochs, batch_size
training_epochs = 15
batch_size = 100
total_iterator = int(mnist.train.num_examples / batch_size)

print(mnist.train.num_examples)
print(total_iterator)

55000
550


In [19]:
# Fit the train
for epoch in range(training_epochs):
    avg_cost = 0
    total_iterator = total_iterator
    
    for i in range(total_iterator):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c_val, _ = session.run([cost, train], feed_dict=feed_dict)
        avg_cost = c_val / batch_size
    
    print("Epoch:", "%04d" % (epoch+1), ", Cost:", "{:.5f}".format(avg_cost))

print('Learning Finished!')

print("Accuracy:", session.run(accuracy,
                               feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 , Cost: 0.00173
Epoch: 0002 , Cost: 0.00054
Epoch: 0003 , Cost: 0.00155
Epoch: 0004 , Cost: 0.00240
Epoch: 0005 , Cost: 0.00113
Epoch: 0006 , Cost: 0.00062
Epoch: 0007 , Cost: 0.00044
Epoch: 0008 , Cost: 0.00196
Epoch: 0009 , Cost: 0.00211
Epoch: 0010 , Cost: 0.00043
Epoch: 0011 , Cost: 0.00042
Epoch: 0012 , Cost: 0.00194
Epoch: 0013 , Cost: 0.00160
Epoch: 0014 , Cost: 0.00138
Epoch: 0015 , Cost: 0.00136
Learning Finished!
Accuracy: 0.9718


### weight을 initializer=tf.contrib.layers.xavier_initializer() 를 놓기만 했는데, 즉, 좀 더 좋은 초기값으로면 바꿨는데, accuracy가 0.9607 --> 0.9718로 증가함!
그리고 초기 Cost 자체가 엄청 작음.
